In [1]:
!nvidia-smi

Tue Dec  7 12:11:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 53%   34C    P8    39W / 340W |    294MiB / 10008MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import torch_geometric
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints
from torch_geometric.transforms import Compose
from torch_geometric.transforms import LinearTransformation
from torch_geometric.transforms import GenerateMeshNormals
from torch_geometric.transforms import NormalizeScale
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from torch_scatter import scatter_mean
import sys

## NOTE:
# Made it work with ModelNet10 (in folder "data/ModelNet") that has only 10 classes
# Now I'll try to test it with ModelNet40 ("data/ModelNet40") with 40 classes

# ----------------------------------------------------------------
# Hyper parameters:
num_points = 1024    # 1024 seems to be the limit...?
batch_size = 32      # not yet used
num_epochs = 100
learning_rate = 0.001
modelnet_num = 10    # 10 or 40


# ----------------------------------------------------------------
# Choosing device:
device = "cuda" if torch.cuda.is_available() else "cpu"


##########################################################################
#                  NO LONGER USED but leave them here...
F = [128, 512, 1024]  # Number of graph convolutional filters.
K = [6, 5, 3]         # Polynomial orders.
M = [512, 128, 10]    # Output dimensionality of fully connected layers.
##########################################################################

# ------------------------------------------------------------------------
## Data loading:
# For ModelNet10 change root to "data/ModelNet"   -> 10 classes
# For MpdelNet40 change root to "data/ModelNet40" -> 40 classes
# Don't forget to change accordingly the output layer from the model...
transforms = Compose([SamplePoints(num_points, include_normals=True), NormalizeScale()])

root = "data/ModelNet"+str(modelnet_num)
dataset_train = ModelNet(root=root, name=str(modelnet_num), train=True, transform=transforms)
dataset_test = ModelNet(root=root, name=str(modelnet_num), train=False, transform=transforms)

# Shuffle Data
dataset_train = dataset_train.shuffle()
dataset_test = dataset_test.shuffle()

# Verification...
print(f"Train dataset shape: {dataset_train}")
print(f"Test dataset shape:  {dataset_test}")

print(dataset_train[0])

Train dataset shape: ModelNet10(3991)
Test dataset shape:  ModelNet10(908)
Data(pos=[1024, 3], y=[1], normal=[1024, 3])


In [3]:
import torch.nn as nn
import torch_geometric.utils as utils
import torch_geometric.nn.conv as conv

class GetGraph(nn.Module):
    def __init__(self):
        """
        Creates the weighted adjacency matrix 'W'
        Taked directly from RGCNN
        """
        super(GetGraph, self).__init__()

    def forward(self, point_cloud):
        point_cloud_transpose = point_cloud.permute(0, 2, 1)
        point_cloud_inner = torch.matmul(point_cloud, point_cloud_transpose)
        point_cloud_inner = -2 * point_cloud_inner
        point_cloud_square = torch.sum(torch.mul(point_cloud, point_cloud), dim=2, keepdim=True)
        point_cloud_square_tranpose = point_cloud_square.permute(0, 2, 1)
        adj_matrix = point_cloud_square + point_cloud_inner + point_cloud_square_tranpose
        adj_matrix = torch.exp(-adj_matrix)
        return adj_matrix


class GetLaplacian(nn.Module):
    def __init__(self, normalize=True):
        """
        Computes the Graph Laplacian from a Weighted Graph
        Taken directly from RGCNN - currently not used - might need to find alternatives in PyG for loss function
        """
        super(GetLaplacian, self).__init__()
        self.normalize = normalize

        def diag(self, mat):
        # input is batch x vertices
            d = []
            for vec in mat:
                d.append(torch.diag(vec))
            return torch.stack(d)

    def forward(self, adj_matrix):
        if self.normalize:
            D = torch.sum(adj_matrix, dim=1)
            eye = torch.ones_like(D)
            eye = self.diag(eye)
            D = 1 / torch.sqrt(D)
            D = self.diag(D)
            L = eye - torch.matmul(torch.matmul(D, adj_matrix), D)
        else:
            D = torch.sum(adj_matrix, dim=1)
            D = torch.diag(D)
            L = D - adj_matrix
        return L

In [4]:
import torch.nn as nn
import torch_geometric.utils as utils
import torch_geometric.nn.conv as conv
import torch

class RGCNN_model(nn.Module):
    def __init__(self, vertice, F, K, M, regularization = 0, dropout = 0):
        # verify the consistency w.r.t. the number of layers
        assert len(F) == len(K)
        super(RGCNN_model, self).__init__()
        '''
        F := List of Convolutional Layers dimensions
        K := List of Chebyshev polynomial degrees
        M := List of Fully Connected Layers dimenstions
        
        Currently the dimensions are 'hardcoded'
        '''
        self.F = F
        self.K = K
        self.M = M

        self.vertice = vertice
        self.regularization = regularization    # gamma from the paper: 10^-9
        self.dropout = dropout
        self.regularizers = []

        # initialize the model layers
        self.get_graph = GetGraph()
        # self.get_laplacian = GetLaplacian(normalize=True)
        self.pool = nn.MaxPool1d(self.vertice)
        self.relu = nn.ReLU()
        self.dropout = torch.nn.Dropout(p=self.dropout)

        ###################################################################
        #                               CHANGE HERE
        self.conv1 = conv.ChebConv(6, 128, 6)
        self.conv2 = conv.ChebConv(128, 512, 5)
        self.conv3 = conv.ChebConv(512, 1024, 3)

        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, modelnet_num)
        ###################################################################

        '''
        for i in range(len(F)):
            if i == 0:
                layer = ChebConv(Fin=3, K=K[i], Fout=F[i])
            else:
                layer = ChebConv(Fin=F[i-1], K=K[i], Fout=F[i])
            setattr(self, 'gcn%d'%i, layer)
        for i in range(len(M)):
            if i==0:
                layer = nn.Linear(F[-1], M[i])
            else:
                layer = nn.Linear(M[i-1], M[i])
            setattr(self, 'fc%d'%i, layer)
        '''

    def forward(self, x):
        self.regularizers = []
        # forward pass
        W   = self.get_graph(x.detach())  # we don't want to compute gradients when building the graph
        edge_index, edge_weight = utils.dense_to_sparse(W)
        out = self.conv1(x, edge_index, edge_weight)
        out = self.relu(out)
        edge_index, edge_weight = utils.remove_self_loops(edge_index, edge_weight)
        L_edge_index, L_edge_weight = torch_geometric.utils.get_laplacian(edge_index.detach(), edge_weight.detach(), normalization="sym")
        L = torch_geometric.utils.to_dense_adj(edge_index=L_edge_index, edge_attr=L_edge_weight)
        self.regularizers.append(torch.linalg.norm(torch.matmul(torch.matmul(torch.Tensor.permute(out.detach(), [0, 2, 1]), L), out.detach())))

        W   = self.get_graph(out.detach())
        edge_index, edge_weight = utils.dense_to_sparse(W)
        out = self.conv2(out, edge_index, edge_weight)
        out = self.relu(out)
        edge_index, edge_weight = utils.remove_self_loops(edge_index, edge_weight)
        L_edge_index, L_edge_weight = torch_geometric.utils.get_laplacian(edge_index.detach(), edge_weight.detach(), normalization="sym")
        L = torch_geometric.utils.to_dense_adj(edge_index=L_edge_index, edge_attr=L_edge_weight)
        self.regularizers.append(torch.linalg.norm(torch.matmul(torch.matmul(torch.Tensor.permute(out.detach(), [0, 2, 1]), L), out.detach())))

        W   = self.get_graph(out.detach())
        edge_index, edge_weight = utils.dense_to_sparse(W)
        out = self.conv3(out, edge_index, edge_weight)
        out = self.relu(out)
        edge_index, edge_weight = utils.remove_self_loops(edge_index, edge_weight)
        L_edge_index, L_edge_weight = torch_geometric.utils.get_laplacian(edge_index.detach(), edge_weight.detach(), normalization="sym")
        L = torch_geometric.utils.to_dense_adj(edge_index=L_edge_index, edge_attr=L_edge_weight)
        self.regularizers.append(torch.linalg.norm(torch.matmul(torch.matmul(torch.Tensor.permute(out.detach(), [0, 2, 1]), L), out.detach())))

        '''
        for i in range(len(self.F)):
            x = getattr(self, 'gcn%d'%i)(x, L)
            print(x)
            x = self.relu(x)
        '''

        out = out.permute(0, 2, 1) # Transpose
        out = self.pool(out)
        out.squeeze_(2)

        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        for param in self.fc1.parameters():
            self.regularizers.append(torch.linalg.norm(param))

        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)
        for param in self.fc1.parameters():
            self.regularizers.append(torch.linalg.norm(param))
        out = self.fc3(out)
        for param in self.fc1.parameters():
            self.regularizers.append(torch.linalg.norm(param))
        '''
        for i in range(len(self.M)):
            x = getattr(self, "fc%d"%i)(x)
        return x
        '''

        return out, self.regularizers
'''
model = RGCNN_model(num_points, F, K, M)

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
'''

'\nmodel = RGCNN_model(num_points, F, K, M)\n\nprint("Model\'s state_dict:")\nfor param_tensor in model.state_dict():\n    print(param_tensor, "\t", model.state_dict()[param_tensor].size())\n\n# Print optimizer\'s state_dict\nprint("Optimizer\'s state_dict:")\nfor var_name in optimizer.state_dict():\n    print(var_name, "\t", optimizer.state_dict()[var_name])\n'

In [ ]:
import os
from datetime import datetime
now = datetime.now()
directory = now.strftime("%d_%m_%y_%H:%M:%S")
parent_directory = "/home/alex/Alex_pyt_geom/Models"
path = os.path.join(parent_directory, directory)
os.mkdir(path)

# Training

# PATH = "/home/alex/Alex_pyt_geom/Models/model"
model_number = 5                # Change this acording to the model you want to load
model = RGCNN_model(num_points, F, K, M, dropout=0.6)

# model.load_state_dict(torch.load(path + '/model' + str(model_number) + '.pt'))

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss = torch.nn.CrossEntropyLoss()
def get_loss(y, labels, regularization, regularizers):
    cross_entropy_loss = loss(y, labels)
    s = torch.sum(torch.as_tensor(regularizers))
    regularization *= s
    l = cross_entropy_loss + regularization
    return l
    
correct_percentage_list = []
model.train()
for epoch in range(num_epochs):
    #dataset_train = dataset_train.shuffle()
    correct = 0
    for i, data in enumerate(dataset_train):
        # make sure the gradients are empty
        optimizer.zero_grad()
        
        # Data preparation 
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred, regularizers = model(x)     # (1 * 40)
        
        class_pred = torch.argmax(y_pred.squeeze(0))  # (1)  
        correct += int((class_pred == y).sum())       # to compute the accuracy for each epoch
        

        # loss and backward
        ###################################################################################
        #                           CrossEntropyLoss
        # This WORKS but I am testing the other way...
        # l = loss(y_pred, y)   # one value
        # l.backward()          # update gradients
        ###################################################################################
       
        l = get_loss(y_pred, y, regularization=1e-9, regularizers=regularizers)
        l.backward()

        # optimisation
        optimizer.step()
        
            
        if i%100==0:
            print(f"Epoch: {epoch}, Sample: {i}, Loss:{l} - Predicted class vs Real Cass: {class_pred} <-> {y.item()}")
            # print(torch.sum(torch.as_tensor(regularizers)))
        if epoch%5==0:
            torch.save(model.state_dict(), path + '/model' + str(epoch) + '.pt')
    print(f"~~~~~~~~~ CORRECT: {correct / len(dataset_train)} ~~~~~~~~~~~")
    correct_percentage_list.append(correct / len(dataset_train))
print(correct_percentage_list)

torch.save(model.state_dict(), "/home/alex/Alex_pyt_geom/Models/final_model.pt")

with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred, _ = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        correct += int((class_pred == y).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")

Epoch: 0, Sample: 0, Loss:2.171449661254883 - Predicted class vs Real Cass: 5 <-> 2
Epoch: 0, Sample: 100, Loss:2.1387202739715576 - Predicted class vs Real Cass: 5 <-> 9
Epoch: 0, Sample: 200, Loss:2.568060874938965 - Predicted class vs Real Cass: 2 <-> 5
Epoch: 0, Sample: 300, Loss:2.3621063232421875 - Predicted class vs Real Cass: 2 <-> 3
Epoch: 0, Sample: 400, Loss:1.9728541374206543 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 0, Sample: 500, Loss:1.2683491706848145 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 0, Sample: 600, Loss:0.46476200222969055 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 0, Sample: 700, Loss:0.7380685210227966 - Predicted class vs Real Cass: 7 <-> 7
Epoch: 0, Sample: 800, Loss:2.010457754135132 - Predicted class vs Real Cass: 7 <-> 1
Epoch: 0, Sample: 900, Loss:1.939794659614563 - Predicted class vs Real Cass: 7 <-> 1
Epoch: 0, Sample: 1000, Loss:2.334803819656372 - Predicted class vs Real Cass: 8 <-> 9
Epoch: 0, Sample: 1100, Loss:0.2670452296733856 

Epoch: 2, Sample: 1200, Loss:0.018450412899255753 - Predicted class vs Real Cass: 1 <-> 1
Epoch: 2, Sample: 1300, Loss:0.6101081967353821 - Predicted class vs Real Cass: 7 <-> 7
Epoch: 2, Sample: 1400, Loss:3.1962859630584717 - Predicted class vs Real Cass: 9 <-> 2
Epoch: 2, Sample: 1500, Loss:7.69734525680542 - Predicted class vs Real Cass: 2 <-> 9
Epoch: 2, Sample: 1600, Loss:0.21501734852790833 - Predicted class vs Real Cass: 7 <-> 7
Epoch: 2, Sample: 1700, Loss:4.63402466266416e-05 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 2, Sample: 1800, Loss:1.6243836879730225 - Predicted class vs Real Cass: 8 <-> 3
Epoch: 2, Sample: 1900, Loss:3.439073324203491 - Predicted class vs Real Cass: 6 <-> 1
Epoch: 2, Sample: 2000, Loss:0.9963369369506836 - Predicted class vs Real Cass: 5 <-> 4
Epoch: 2, Sample: 2100, Loss:0.31989002227783203 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 2, Sample: 2200, Loss:2.0056054381711874e-06 - Predicted class vs Real Cass: 5 <-> 5
Epoch: 2, Sample: 2300, L

Epoch: 4, Sample: 2300, Loss:0.5051628351211548 - Predicted class vs Real Cass: 6 <-> 6
Epoch: 4, Sample: 2400, Loss:1.299087405204773 - Predicted class vs Real Cass: 7 <-> 0
Epoch: 4, Sample: 2500, Loss:2.9952866498206276e-06 - Predicted class vs Real Cass: 5 <-> 5
Epoch: 4, Sample: 2600, Loss:1.7407385257683927e-06 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 4, Sample: 2700, Loss:0.0002661551407072693 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 4, Sample: 2800, Loss:0.0133324870839715 - Predicted class vs Real Cass: 4 <-> 4
Epoch: 4, Sample: 2900, Loss:3.7547526359558105 - Predicted class vs Real Cass: 3 <-> 6
Epoch: 4, Sample: 3000, Loss:6.243284702301025 - Predicted class vs Real Cass: 7 <-> 1
Epoch: 4, Sample: 3100, Loss:1.185569405555725 - Predicted class vs Real Cass: 6 <-> 6
Epoch: 4, Sample: 3200, Loss:0.0003450381918810308 - Predicted class vs Real Cass: 8 <-> 8
Epoch: 4, Sample: 3300, Loss:1.3929448127746582 - Predicted class vs Real Cass: 6 <-> 6
Epoch: 4, Sample: 340

Epoch: 6, Sample: 3400, Loss:4.048899427289143e-06 - Predicted class vs Real Cass: 5 <-> 5
Epoch: 6, Sample: 3500, Loss:0.9636327624320984 - Predicted class vs Real Cass: 3 <-> 8
Epoch: 6, Sample: 3600, Loss:1.5676471321057761e-06 - Predicted class vs Real Cass: 7 <-> 7
Epoch: 6, Sample: 3700, Loss:2.8768647553079063e-06 - Predicted class vs Real Cass: 5 <-> 5
Epoch: 6, Sample: 3800, Loss:3.1171898626780603e-06 - Predicted class vs Real Cass: 5 <-> 5
Epoch: 6, Sample: 3900, Loss:0.2079339474439621 - Predicted class vs Real Cass: 8 <-> 8
~~~~~~~~~ CORRECT: 0.7810072663492859 ~~~~~~~~~~~
Epoch: 7, Sample: 0, Loss:3.869297870551236e-06 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 7, Sample: 100, Loss:0.1325138807296753 - Predicted class vs Real Cass: 9 <-> 9
Epoch: 7, Sample: 200, Loss:2.1805703909194563e-06 - Predicted class vs Real Cass: 5 <-> 5
Epoch: 7, Sample: 300, Loss:0.7834551930427551 - Predicted class vs Real Cass: 3 <-> 3
Epoch: 7, Sample: 400, Loss:2.0924816131591797 - Predi

Epoch: 9, Sample: 500, Loss:0.42821091413497925 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 9, Sample: 600, Loss:2.784862999760662e-06 - Predicted class vs Real Cass: 2 <-> 2
Epoch: 9, Sample: 700, Loss:1.615988480807573e-06 - Predicted class vs Real Cass: 7 <-> 7
Epoch: 9, Sample: 800, Loss:0.001014389330521226 - Predicted class vs Real Cass: 1 <-> 1
Epoch: 9, Sample: 900, Loss:0.00017559988191351295 - Predicted class vs Real Cass: 1 <-> 1
Epoch: 9, Sample: 1000, Loss:3.350326733198017e-05 - Predicted class vs Real Cass: 9 <-> 9
Epoch: 9, Sample: 1100, Loss:1.4716670193593018e-06 - Predicted class vs Real Cass: 7 <-> 7
Epoch: 9, Sample: 1200, Loss:2.503867335690302e-06 - Predicted class vs Real Cass: 1 <-> 1
Epoch: 9, Sample: 1300, Loss:2.045642614364624 - Predicted class vs Real Cass: 0 <-> 7
Epoch: 9, Sample: 1400, Loss:3.920804023742676 - Predicted class vs Real Cass: 4 <-> 2
Epoch: 9, Sample: 1500, Loss:8.357370376586914 - Predicted class vs Real Cass: 2 <-> 9
Epoch: 9, Sample:

Epoch: 11, Sample: 1600, Loss:0.00020793650764971972 - Predicted class vs Real Cass: 7 <-> 7
Epoch: 11, Sample: 1700, Loss:0.9411947727203369 - Predicted class vs Real Cass: 9 <-> 2
Epoch: 11, Sample: 1800, Loss:1.41150963306427 - Predicted class vs Real Cass: 3 <-> 3
Epoch: 11, Sample: 1900, Loss:1.8631551265716553 - Predicted class vs Real Cass: 3 <-> 1
Epoch: 11, Sample: 2000, Loss:0.4412958025932312 - Predicted class vs Real Cass: 4 <-> 4
Epoch: 11, Sample: 2100, Loss:1.4096112251281738 - Predicted class vs Real Cass: 3 <-> 8
Epoch: 11, Sample: 2200, Loss:6.666233275609557e-06 - Predicted class vs Real Cass: 5 <-> 5
Epoch: 11, Sample: 2300, Loss:0.8703680634498596 - Predicted class vs Real Cass: 6 <-> 6
Epoch: 11, Sample: 2400, Loss:3.361557173775509e-05 - Predicted class vs Real Cass: 0 <-> 0
Epoch: 11, Sample: 2500, Loss:5.145224804437021e-06 - Predicted class vs Real Cass: 5 <-> 5
Epoch: 11, Sample: 2600, Loss:2.9226716833363753e-06 - Predicted class vs Real Cass: 2 <-> 2
Epoch:

In [ ]:
print(correct_percentage_list)

In [ ]:
with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        # print("Pred: ", class_pred.item(), "Real: " , y.item())
        correct += int((class_pred == y).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")

In [ ]:
with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred, _ = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        print("Pred: ", class_pred.item(), "Real: " , y.item())
        correct += int((class_pred == y_pred).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")
    

In [ ]:
with torch.no_grad():
    x = torch.rand([1, 1024, 6])*5+3
    x = x.to(device)
    y = model(x)
    y_class = torch.argmax(y)
    print(y)
    print(y_class)

WORK IN PROGRESS!!! 
Trying to create batches from data...

In [ ]:
import numpy as np
from torch_geometric.data import Data

length = len(dataset_train)
batch_size = 32
iterations = np.ceil(length/batch_size)
iterations = iterations.astype(int)
batched_data = torch.empty([125, 1024, 6])
print(dataset_train)
print(dataset_train[0])
aux = Data()
for i in range(iterations):
    ob = dataset_train[i:i+batch_size]
    pos=torch.empty([0, 3])
    y = torch.empty([0])
    normal = torch.empty([0, 3])
    for data in ob:
        pos = torch.cat([pos, data.pos])
        y = torch.cat([y, data.y])
        normal = torch.cat([normal, data.normal])
    batch_data[i] = Data(pos=pos, y=y, normal=normal)
print(pos.shape)

In [ ]:
#print(pos.shape)
#print(pos)
print(len(batch_data))
Batched_data = torch.empty([125, 1024, 6])
BATCHED_DATA = []
for i in range(125):
    # print(batch_data[i].pos)
    pos = torch.empty([32, 1024, 3])
    y = torch.empty([32, 1])
    normal = torch.empty([32, 1024, 3])
    for i in range(batch_size):
        pos[i] = batch_data[i].pos[num_points*i:num_points*i+1024]
        y[i] = batch_data[i].y[i]
        normal[i] = batch_data[i].normal[num_points*i:num_points*i+1024]
    BATCH = Data(pos=pos, y=y, normal=normal)
    BATCHED_DATA.append(BATCH)
    # Batched_data[i] = Data(pos=pos, y=y, normal=normal)
print(pos.shape)
print(normal.shape)
print(y.shape)
print(len(BATCHED_DATA))
for data in BATCHED_DATA:
    print(data)